In [1]:
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

In [2]:
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

In [3]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [4]:
# Design variables
center_x =0
center_y =0
x = 0.6875+0.650 # qubit x spacing
y = 0.8+0.650 # qubit y spacing
offset = x//2

feed_x = x*1.5

In [5]:
design.delete_all_components()

In [7]:
# Qubits
connection_pads = dict(B0 = dict(loc_W=-1, loc_H=-1, pad_width='120um'),
                       B1  = dict(loc_W=-1, loc_H=+1, pad_width='120um'),
                       B2  = dict(loc_W=+1, loc_H=-1, pad_width='120um'),
                       B3 = dict(loc_w = +1, loc_H = +1, pad_width='120um'))

space=[y/2,-y/2,-3*y/2]                           
q0 = TransmonPocket(design, "Q0", options=dict(pos_x=f"{center_x-2*x}mm",
                                               pos_y=f"{center_y+2*y}mm",
                                               chip="Q_chip",
                                               connection_pads=connection_pads))
for i,j in zip(range(1,4), space):
    a=[TransmonPocket(design,'Q'+str(i), options=dict(pos_x=f"{center_x-2*x}mm",
                                                    pos_y=f"{center_y+j*y}",
                                                    chip="Q_chip",
                                               connection_pads=connection_pads))]

q4 = TransmonPocket(design, "Q4", options=dict(pos_x=f"{center_x-x}mm",
                                               pos_y=f"{center_y+2*y}mm",
                                               chip="Q_chip",
                                               connection_pads=connection_pads))
for i,j in zip(range(1,4),space) :
    a=[TransmonPocket(design,'Q'+str(i+4), options=dict(pos_x=f"{center_x-x}mm",
                                                    pos_y=f"{center_y+j*y}",
                                                    chip="Q_chip",
                                               connection_pads=connection_pads))]                                               
q8 = TransmonPocket(design, "Q8", options=dict(pos_x=f"{center_x}mm",
                                               pos_y=f"{center_y+2*y}mm",
                                               chip="Q_chip",
                                               connection_pads=connection_pads))
for i,j in zip(range(1,4),space) :
    a=[TransmonPocket(design,'Q'+str(i+8), options=dict(pos_x=f"{center_x}mm",
                                                    pos_y=f"{center_y+j*y}",
                                                    chip="Q_chip",
                                               connection_pads=connection_pads))]
q12 = TransmonPocket(design, "Q12", options=dict(pos_x=f"{center_x+x}mm",
                                               pos_y=f"{center_y+2*y}mm",
                                               chip="Q_chip",
                                               connection_pads=connection_pads))
for i,j in zip(range(1,4),space) :
    a=[TransmonPocket(design,'Q'+str(i+12), options=dict(pos_x=f"{center_x+x}mm",
                                                    pos_y=f"{center_y+j*y}",
                                                    chip="Q_chip",
                                               connection_pads=connection_pads))]                                               
q16 = TransmonPocket(design, "Q16", options=dict(pos_x=f"{center_x+2*x}mm",
                                               pos_y=f"{center_y+2*y}mm",
                                               chip="Q_chip",
                                               connection_pads=connection_pads))
for i,j in zip(range(1,4),space) :
    a=[TransmonPocket(design,'Q'+str(i+16), options=dict(pos_x=f"{center_x+2*x}mm",
                                                    pos_y=f"{center_y+j*y}",
                                                    chip="Q_chip",
                                               connection_pads=connection_pads))]

design.rebuild()
gui.rebuild()
gui.autoscale()

In [16]:
# Feedline Launchpads
q0_xpos=float(q0.options.pos_x.strip('mm'))
q0_ypos=float(q0.options.pos_y.strip('mm'))
q16_xpos=float(q16.options.pos_x.strip('mm'))
q16_ypos=float(q16.options.pos_y.strip('mm'))
flpad1 = LaunchpadWirebondCoupled(design, "L1",
                                    options=dict(pos_x=f'{q0_xpos-1}mm',
                                                 pos_y=f'{q0_ypos+-(y/2)}mm',
                                                 chip="C_chip",
                                                 orientation="0"))

flpad2 = LaunchpadWirebondCoupled(design, "L2",
                                     options=dict(pos_x=f'{q16_xpos+1}mm',
                                                  pos_y=f'{q16_ypos+-(y/2)}mm',
                                                  chip="C_chip",
                                                  orientation="180"))
flpad3 = LaunchpadWirebondCoupled(design, "L3",
                                    options=dict(pos_x=f'{q0_xpos-1}mm',
                                                 pos_y=f'{q0_ypos-7+0.35}mm',
                                                 chip="C_chip",
                                                 orientation="90"))

flpad4 = LaunchpadWirebondCoupled(design, "L4",
                                     options=dict(pos_x=f'{q16_xpos+1}mm',
                                                  pos_y=f'{q16_ypos-7+0.35}mm',
                                                  chip="C_chip",
                                                  orientation="90"))


design.rebuild()
gui.rebuild()
gui.autoscale()

In [10]:
# Feedline
pin_opt1 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='L1',
                                    pin='tie'),
                                end_pin=Dict(
                                    component='L2',
                                    pin='tie')),
                                    chip="C_chip",
                                    fillet='99um')
pin_opt1.lead.start_straight = '1.975mm'
pin_opt1.lead.end_straight = '1.975mm'
pin_opt2 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='L3',
                                    pin='tie'),
                                end_pin=Dict(
                                    component='L4',
                                    pin='tie')),
                                    chip="C_chip",
                                    fillet='99um')
pin_opt2.lead.start_straight = '1.975mm'
pin_opt2.lead.end_straight = '1.975mm'
fl1 = RouteStraight(design, "fl1", pin_opt1)
fl1 = RouteStraight(design, "fl2", pin_opt2)
design.rebuild()
gui.rebuild()
gui.autoscale()                   


In [40]:
fl1.default_options

{'pin_inputs': {'start_pin': {'component': '', 'pin': ''},
  'end_pin': {'component': '', 'pin': ''}},
 'fillet': '0',
 'lead': {'start_straight': '0mm',
  'end_straight': '0mm',
  'start_jogged_extension': '',
  'end_jogged_extension': ''},
 'total_length': '7mm',
 'trace_width': 'cpw_width'}

In [13]:
cp1top = LaunchpadWirebondCoupled(design, "cp1",
                                    options=dict(pos_x=f'{q0_xpos-0.3}mm',
                                                 pos_y=f'{q0_ypos+1-0.15}mm',
                                                 chip="C_chip",
                                                 orientation="270"))

cp2top = LaunchpadWirebondCoupled(design, "cp2",
                                     options=dict(pos_x=f'{q0_xpos-0.3+x}mm',
                                                  pos_y=f'{q16_ypos+1-0.15}mm',
                                                  chip="C_chip",
                                                  orientation="270"))
cp3top = LaunchpadWirebondCoupled(design, "cp3",
                                    options=dict(pos_x=f'{q0_xpos+0.3+3*x}mm',
                                                 pos_y=f'{q0_ypos+1-0.15}mm',
                                                 chip="C_chip",
                                                 orientation="270"))

cp4top = LaunchpadWirebondCoupled(design, "cp4",
                                     options=dict(pos_x=f'{q0_xpos+0.3+4*x}mm',
                                                  pos_y=f'{q16_ypos+1-0.15}mm',
                                                  chip="C_chip",
                                                  orientation="270"))
design.rebuild()
gui.rebuild()
gui.autoscale()                   


In [26]:
cp5bot = LaunchpadWirebondCoupled(design, "cp1",
                                    options=dict(pos_x=f'{q0_xpos-0.3}mm',
                                                 pos_y=f'{q0_ypos-7}mm',
                                                 chip="C_chip",
                                                 orientation="90"))

cp6bot= LaunchpadWirebondCoupled(design, "cp6",
                                     options=dict(pos_x=f'{q0_xpos-0.3+x}mm',
                                                  pos_y=f'{q16_ypos-7}mm',
                                                  chip="C_chip",
                                                  orientation="90"))
cp7bot = LaunchpadWirebondCoupled(design, "cp7",
                                    options=dict(pos_x=f'{q0_xpos+0.3+3*x}mm',
                                                 pos_y=f'{q0_ypos+-7}mm',
                                                 chip="C_chip",
                                                 orientation="90"))

cp8bot = LaunchpadWirebondCoupled(design, "cp8",
                                     options=dict(pos_x=f'{q0_xpos+0.3+4*x}mm',
                                                  pos_y=f'{q16_ypos+-7}mm',
                                                  chip="C_chip",
                                                  orientation="90"))
design.rebuild()
gui.rebuild()
gui.autoscale()                   


In [34]:
pin_opt1 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='cp1',
                                    pin='tie'),
                                end_pin=Dict(
                                    component='Q0',
                                    pin='B1')),
                                    chip="C_chip",
                                    total_length='0.5mm',
                                    fillet='99um')

pin_opt2 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='cp2',
                                    pin='tie'),
                                end_pin=Dict(
                                    component='Q4',
                                    pin='B1')),
                                    chip="C_chip",
                                    total_length='0.5mm',
                                    fillet='99um')
pin_opt3 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='cp3',
                                    pin='tie'),
                                end_pin=Dict(
                                    component='Q12',
                                    pin='B3')),
                                    chip="C_chip",
                                    total_length='0.5mm',
                                    fillet='99um')

pin_opt4 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='cp4',
                                    pin='tie'),
                                end_pin=Dict(
                                    component='Q16',
                                    pin='B3')),
                                    chip="C_chip",
                                    total_length='0.5mm',
                                    fillet='99um')                                    

cl1 = RouteMeander(design, "cl1", pin_opt1)
cl2 = RouteMeander(design, "cl2", pin_opt2)
cl3 = RouteMeander(design, "cl3", pin_opt3)
cl4 = RouteMeander(design, "cl4", pin_opt4)
design.rebuild()
gui.rebuild()
gui.autoscale()                   


10:05PM 43s WARNING [check_lengths]: For path table, component=cl1, key=trace has short segments that could cause issues with fillet. Values in (1-5)  are index(es) in shapely geometry.
10:05PM 43s WARNING [check_lengths]: For path table, component=cl1, key=cut has short segments that could cause issues with fillet. Values in (1-5)  are index(es) in shapely geometry.
10:05PM 43s WARNING [check_lengths]: For path table, component=cl2, key=trace has short segments that could cause issues with fillet. Values in (1-5)  are index(es) in shapely geometry.
10:05PM 43s WARNING [check_lengths]: For path table, component=cl2, key=cut has short segments that could cause issues with fillet. Values in (1-5)  are index(es) in shapely geometry.
10:05PM 43s WARNING [check_lengths]: For path table, component=cl3, key=trace has short segments that could cause issues with fillet. Values in (1-6)  are index(es) in shapely geometry.
10:05PM 43s WARNING [check_lengths]: For path table, component=cl3, key=cu

: 

In [ ]:
q0 = TransmonPocket(design, "Q0", options=dict(pos_x=f"{center_x-2*x}mm",
                                               pos_y=f"{center_y+2*y}mm",
                                               chip="Q_chip",
                                               connection_pads=connection_pads))

In [31]:
cl1.options

{'chip': 'C_chip',
 'layer': '1',
 'pin_inputs': {'start_pin': {'component': 'cp1', 'pin': 'tie'},
  'end_pin': {'component': 'Q0', 'pin': 'B1'}},
 'fillet': '0',
 'lead': {'start_straight': '0mm',
  'end_straight': '0mm',
  'start_jogged_extension': '',
  'end_jogged_extension': ''},
 'total_length': '7mm',
 'trace_width': 'cpw_width',
 'meander': {'spacing': '200um', 'asymmetry': '0um'},
 'snap': 'true',
 'prevent_short_edges': 'true',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False,
 'trace_gap': 'cpw_gap',
 '_actual_length': '7.000000000000004 mm'}

In [49]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [50]:
c1.sim.setup.name = 'LOM'
c1.sim.setup.max_passes = 14
c1.sim.setup


{'name': 'LOM',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': 5.0,
 'save_fields': False,
 'enabled': True,
 'max_passes': 14,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': True,
 'solution_order': 'High',
 'solver_type': 'Iterative'}

In [51]:
c1.sim.run(components=['Q1'])

INFO 11:20AM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:20AM [load_ansys_project]: 	Opened Ansys App
INFO 11:20AM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 11:20AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project11
INFO 11:20AM [connect_design]: No active design found (or error getting active design).
INFO 11:20AM [connect]: 	 Connected to project "Project11". No design detected
INFO 11:20AM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
WARNING 11:20AM [connect_setup]: 	No design setup detected.
WARNING 11:20AM [connect_setup]: 	Creating Q3D default setup.
INFO 11:20AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 11:20AM [get_setup]: 	Opened setup `LOM`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 11:20AM [analyze]: Analyzing setup LOM
INFO 11:22AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\

In [7]:
c1.close

NameError: name 'c1' is not defined

In [52]:
c1.setup.junctions = Dict({'Lj': 12.31, 'Cj': 2})
c1.setup.freq_readout = 6.6
#c1.setup.freq_bus = [6.0, 6.2,6.4]
c1.run_lom()

ValueError: Capacitance matrix is not the right size

In [45]:
from qiskit_metal.renderers.renderer_gmsh.gmsh_renderer import QGmshRenderer


ModuleNotFoundError: No module named 'qiskit_metal.renderers.renderer_gmsh'

In [2]:
design = designs.DesignPlanar({}, True)
design.chips.main.size.size_x = '8mm'
design.chips.main.size.size_y = '8mm'
design.chips.main.material = 'Silicon'
gui = MetalGUI(design)

In [5]:
N_x =4
N_y =3

In [7]:
#Loop to generate and draw the qubits
for x in range(N_x):
    for y in range(N_y):
        options = dict(pos_x= str(x*3000)+'um', pos_y = str(y*3000 + (x%2)*1500)+'um', orientation = "-90",
                      connection_pads = dict(
                            B0 = dict(loc_W=-1, loc_H=-1, pad_width='75um'),
                            B1  = dict(loc_W=-1, loc_H=+1, pad_width='120um'),
                            B2  = dict(loc_W=+1, loc_H=-1, pad_width='120um'),
                            B3 = dict(loc_w = +1, loc_H = +1, pad_width='90um')))


        obj=TransmonPocket6(design,'Q_'+str(x)+'_'+str(y),options)


gui.rebuild()
gui.autoscale()

In [12]:
for x in range(N_x):
    for y in range(N_y):
        #"upward" connection, avoids drawing connectors for 'top' row. Changes connector length by +/-50um to avoid frequency collisions
        if y<(N_y-1):
            connectorAD = RouteMeander(design,'CU_'+str(x)+'_'+str(y),options = dict(total_length = str(7+(y%2)*0.5)+'mm',
                                                                                    fillet = "99um",lead=dict(
                                                                                    start_straight='0.5mm',
                                                                                    end_straight='0.25mm'),
                                                                                    meander = dict(asymmetry='-700um'),
                                                                                    pin_inputs = dict(
                                                                                        start_pin=dict(
                                                                                            component ='Q_'+str(x)+'_'+str(y),
                                                                                            pin = 'B0'),
                                                                                        end_pin=dict(
                                                                                            component='Q_'+str(x)+'_'+str(y+1),
                                                                                            pin='B3'))))

        #"sideways" connection, avoids drawing for far right col, and for top qubit in odd col. Changes connector length by +/- 25um
        #to avoid frequency collisions
        if x<(N_x-1) and (not(x%2 and y==(N_y-1))):
            connectorBC = RouteMeander(design,'CS_'+str(x)+'_'+str(y),options= dict(total_length = str(6+(y%2)*0.5)+'mm',
                                                                                    fillet = "99um",lead=Dict(
                                                                                        start_straight='0.3mm',
                                                                                        end_straight='0.25mm'),
                                                                                    meander = Dict(asymmetry='-200um'),
                                                                                    pin_inputs = Dict(
                                                                                        start_pin=Dict(
                                                                                            component ='Q_'+str(x)+'_'+str(y),
                                                                                            pin = 'B1'),
                                                                                        end_pin=Dict(
                                                                                            component='Q_'+str(x+1)+'_'+str(y+(x%2)),
                                                                                             pin='B2'))))


gui.rebuild()
gui.autoscale()

In [13]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [14]:
c1.sim.setup.name = 'LOM'
c1.sim.setup.max_passes = 14

#To change multiple settings use the following method:
#c1.sim.setup_update(name = 'LOM', max_passes = 14)

c1.sim.setup

{'name': 'LOM',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': 5.0,
 'save_fields': False,
 'enabled': True,
 'max_passes': 14,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': True,
 'solution_order': 'High',
 'solver_type': 'Iterative'}

In [15]:
c1.sim.run(components=['Q_0_0'], open_terminations=[('Q_0_0', 'B0'), ('Q_0_0', 'B1'), ('Q_0_0', 'B2'), ('Q_0_0', 'B3')])

INFO 12:19PM [connect_project]: Connecting to Ansys Desktop API...
INFO 12:19PM [load_ansys_project]: 	Opened Ansys App
INFO 12:19PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 12:19PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project10
INFO 12:19PM [connect_design]: No active design found (or error getting active design).
INFO 12:19PM [connect]: 	 Connected to project "Project10". No design detected
INFO 12:19PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
WARNING 12:19PM [connect_setup]: 	No design setup detected.
WARNING 12:19PM [connect_setup]: 	Creating Q3D default setup.
INFO 12:19PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 12:19PM [get_setup]: 	Opened setup `LOM`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 12:19PM [analyze]: Analyzing setup LOM
INFO 12:20PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\

In [16]:
c1.setup.junctions = Dict({'Lj': 12.31, 'Cj': 2})
c1.setup.freq_readout = 6.6
c1.setup.freq_bus = [6.0, 6.2,6.4]

c1.run_lom()

[4, 5] [6 0 1 2]
Predicted Values

Transmon Properties
f_Q 4.754584 [GHz]
EC 235.617680 [MHz]
EJ 13.273404 [GHz]
alpha -267.930318 [MHz]
dispersion 3.181384 [KHz]
Lq 12.305036 [nH]
Cq 82.210418 [fF]
T1 41.892238 [us]

**Coupling Properties**

tCqbus1 -14.168944 [fF]
gbus1_in_MHz -163.900064 [MHz]
χ_bus1 -3.805361 [MHz]
1/T1bus1 3651.373801 [Hz]
T1bus1 43.587688 [us]

tCqbus2 0.996945 [fF]
gbus2_in_MHz 10.593391 [MHz]
χ_bus2 -0.032439 [MHz]
1/T1bus2 3.393832 [Hz]
T1bus2 46895.349811 [us]

tCqbus3 7.265566 [fF]
gbus3_in_MHz 79.562015 [MHz]
χ_bus3 -1.398675 [MHz]
1/T1bus3 141.550333 [Hz]
T1bus3 1124.369963 [us]

tCqbus4 -1.135063 [fF]
gbus4_in_MHz -12.841880 [MHz]
χ_bus4 -0.028798 [MHz]
1/T1bus4 2.833115 [Hz]
T1bus4 56176.667488 [us]
Bus-Bus Couplings
gbus1_2 30.395446 [MHz]
gbus1_3 54.018528 [MHz]
gbus1_4 37.602545 [MHz]
gbus2_3 12.735149 [MHz]
gbus2_4 17.547679 [MHz]
gbus3_4 15.159161 [MHz]


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.067386,269.734507,13.273404,-310.238448,12.172869,"[-152.6813549370898, 7.884817440873428, 69.643...","[-5.230385414450512, -0.03360430526167508, -1....",5.230385,152.681355
2,4.978952,259.82427,13.273404,-297.862565,8.475755,"[-138.64426863269293, 4.5331404478046675, 71.5...","[-3.7689499205004067, -0.009194793777363161, -...",3.768950,138.644269
3,4.892896,250.381742,13.273404,-286.136558,5.883582,"[-146.88367486844987, 7.4375764026439155, 73.1...","[-3.7243908417064624, -0.020797507589197, -1.4...",3.724391,146.883675
4,4.866873,247.56522,13.273404,-282.651319,5.255073,"[-152.11652111536472, 6.9265321076437525, 74.3...","[-3.846208966429142, -0.01714172085390088, -1....",3.846209,152.116521
5,4.829861,243.590335,13.273404,-277.74237,4.465526,"[-154.92585342951307, 8.309105352795504, 75.98...","[-3.782563866916498, -0.0229721453783535, -1.4...",3.782564,154.925853
6,4.803601,240.792178,13.273404,-274.293459,3.972245,"[-157.31217332018088, 9.226811897601223, 76.86...","[-3.756625064730378, -0.02695363248825837, -1....",3.756625,157.312173
7,4.788301,239.170225,13.273404,-272.296862,3.70813,"[-160.47986285976032, 9.960968235448897, 77.93...","[-3.8255528890029233, -0.030526566325938972, -...",3.825553,160.479863
8,4.772932,237.547206,13.273404,-270.300838,3.458947,"[-161.27108680950445, 9.751667401738398, 78.53...","[-3.7804634905817007, -0.028433738604866796, -...",3.780463,161.271087
9,4.763217,236.524421,13.273404,-269.043963,3.309358,"[-162.37247327663806, 10.002730529373782, 78.9...","[-3.7802650968333644, -0.029384903013254006, -...",3.780265,162.372473
10,4.754584,235.61768,13.273404,-267.930318,3.181384,"[-163.9000635746779, 10.59339149645099, 79.562...","[-3.805360546965784, -0.0324389720895161, -1.3...",3.805361,163.900064
